mas temas de sql

In [2]:
import pandas as pd
import sqlite3 as sql 
from pym import tools as t

# 1. DISTINCT: se le aplica a una columna
# a. Valores unicos de esa columna
# b. Eliminar duplicados

# Selecciona TODAS las columnas de la tabla ventas
query = """
SELECT 
    --- Esto es un comentario en SQL
    * --- TODAS LAS COLUMNAS
FROM 
    ventas
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-01,Cargadores,HUI-844,39,3954.10,154209.90,Milpa Alta
1,2025-01-01,Cuadernos,FOY-846,47,3444.24,161879.28,Santa Fé
2,2025-01-01,Papel,QTJ-739,10,506.73,5067.30,Centro
3,2025-01-01,Audífonos,UHM-612,26,6529.70,169772.20,Centro
4,2025-01-01,Borradores,EWV-583,31,7461.07,231293.17,Polanco
...,...,...,...,...,...,...,...
2995,2025-01-03,Cajas,LID-283,14,6280.82,87931.48,Santa Fé
2996,2025-01-03,Pinturas,VFA-558,5,349.92,1749.60,Centro
2997,2025-01-03,Pegamento,UMR-274,28,1403.66,39302.48,Cuemanco
2998,2025-01-03,Pc,RKY-462,38,216.58,8230.04,Coapa


In [3]:
# Order by solito, el ordenamiento es de manera ascendente
query = """
SELECT 
    DISTINCT Fecha
FROM 
    ventas
--- Sentencia de ordenamiento
ORDER BY
    Fecha DESC
--- limit n, te muestra las primeras n filas
LIMIT 5
"""
t.consultar_bbdd(query)

,Fecha
0,2025-01-03
1,2025-01-02
2,2025-01-01


In [5]:
# Las 5 primeras ventas del 6 de enero en Polanco
# WHERE ---> filtros

# Seleccioname todas las columnas de la tabla ventas
# donde la sucursal sea Polando Y el dia sea el 6 de enero
#                               &
#                              and
# o ---> | ----> or
query = """
SELECT 
    Producto, 
    Cantidad,
    Total AS Total_Ventas
FROM 
    ventas
WHERE
    Sucursal = 'Polanco'
AND 
    Fecha = '2025-01-01'
LIMIT 5
"""
t.consultar_bbdd(query)

,Producto,Cantidad,Total_Ventas
0,Borradores,31,231293.17
1,Tablets,33,39628.71
2,Lápices,32,71732.16
3,Pc,2,202.82
4,Pinturas,10,59046.20


Jerarquía de instrucciones en SQL

SELECT
FROM
WHERE
ORDER BY
LIMIT 5
Funciones de agregación:

SUM
MIN/MAX
AVG
COUNT

In [6]:
# CAST() ---> te convierte una columna al tipo de dato que le indiques
# CAST(NOM_COL AS TIPO_DATO_NUEVO)
query = """
SELECT 
    CAST(SUM(Total) AS INTEGER) AS Total_Ventas_Enero_Global
FROM 
    ventas
"""
t.consultar_bbdd(query)

,Total_Ventas_Enero_Global
0,186650986


Definir una función en Python
Subir a SQL

In [7]:
def separador_miles(numero):
    """
    Da formato a un número insertando separadores de miles.

    Parámetros:
    ----------
    numero : int o float
        El número al que se le desea aplicar el formato con separadores de miles.

    Retorna:
    -------
    str
        El número formateado como cadena con comas separando los miles.
    """
    return "{:,}".format(numero)

sep_miles = lambda numero: "{:,}".format(numero)

print(separador_miles(1000))
print(sep_miles(1000))

1,000
1,000


In [8]:
# Conectarse a la base de datos
con = sql.connect("ventas.db")
cursor = con.cursor()

# a. nombre de la funcion en SQL
# b. cuantos parametros tiene tu funcion
# c. pones la funcion de Python
con.create_function("separador_miles", 1, separador_miles)

query = """
SELECT 
    separador_miles(
        CAST(
            SUM(Total) 
        AS 
            INTEGER
        )
    ) AS Total_Ventas_Enero_Global
FROM 
    ventas
"""

# fetchone() ---> nos muestra solo un valor
consulta_01 = cursor.execute(query).fetchone()

# Guardamos cambios
con.commit()
# Cerramos la conexion
con.close()

consulta_01

('186,650,986',)

No puedes usar funciones creadas por ti mism@ con pandas, entonces debemos ver una alternativa

In [9]:
# Conectarse a la base de datos
con = sql.connect("ventas.db")
cursor = con.cursor()

# a. nombre de la funcion en SQL
# b. cuantos parametros tiene tu funcion
# c. pones la funcion de Python
con.create_function("separador_miles", 1, separador_miles)

query = """
SELECT 
    Producto, 
    Cantidad,
    Total AS Total_Ventas
FROM 
    ventas
WHERE
    Sucursal = 'Polanco'
AND 
    Fecha = '2025-01-06'
"""

# fetchone() ---> nos muestra solo un valor
# fetchall() ---> todas las filas disponibles
# fetchmany(n) ---> traeme los primeros n registros
consulta_02 = cursor.execute(query).fetchmany(5)

# Guardamos cambios
con.commit()
# Cerramos la conexion
con.close()

consulta_02

[]


##Resumen de códigos
WHERE: Realizar filtros. Va combinada usualmente con operadores lógicos (AND, OR).
ORDER BY: Realizar el ordenamiento de la tabla con base en una columna. Por defecto se ordena de manera ascendente. DESC, ordenas de manera descendente.
DISTINCT: elimina duplicados, te arroja los valores únicos de una columna.
LIMIT N: te muestra los primeros N registros
OFFSET: salto de página
LIMIT-OFFSET: paginación

In [10]:

query = """
SELECT 
    *
FROM 
    ventas
LIMIT 15
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-01,Cargadores,HUI-844,39,3954.10,154209.90,Milpa Alta
1,2025-01-01,Cuadernos,FOY-846,47,3444.24,161879.28,Santa Fé
2,2025-01-01,Papel,QTJ-739,10,506.73,5067.30,Centro
3,2025-01-01,Audífonos,UHM-612,26,6529.70,169772.20,Centro
4,2025-01-01,Borradores,EWV-583,31,7461.07,231293.17,Polanco
5,2025-01-01,Pegamento,ECV-177,12,6431.01,77172.12,CU
6,2025-01-01,Bolsas,YSA-138,42,782.29,32856.18,Santa Fé
7,2025-01-01,Monitores,SME-278,38,307.89,11699.82,Milpa Alta
8,2025-01-01,Pegamento,FEJ-643,11,1356.22,14918.42,Xochimilco
9,2025-01-01,Teclados,VRF-593,26,2134.93,55508.18,Xochimilco


In [11]:
query = """
SELECT 
    *
FROM 
    ventas
LIMIT 5
OFFSET 10
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-01,Tablets,TXF-934,33,1200.87,39628.71,Polanco
1,2025-01-01,Pegamento,VIN-287,14,680.00,9520.00,Centro
2,2025-01-01,Sacapuntas,DMT-425,14,3434.53,48083.42,Narvarte
3,2025-01-01,Bolsas,NHZ-584,16,151.82,2429.12,Milpa Alta
4,2025-01-01,Cajas,WGC-148,22,2840.51,62491.22,Milpa Alta
